In [1]:
from mixclu import *

from IPython.display import display
pd.options.display.max_columns = None

In [2]:
df     = pd.read_csv('./datasets/church.txt')
labels = df.pop('labels')

In [ ]:
df.head(5)

In [3]:
id_col, cat_col, con_col = get_types(df)
con_col.extend(id_col)

In [4]:
scores      = pd.DataFrame()
predictions = pd.DataFrame()
predictions['Label'] = labels
data        = df

In [12]:
result_a, matrix_a = famd_kemans(df, 
                cat_col, 
                clusters = 4, 
                random_state = 32, 
                max_iter = 300, 
                verbose = 0, 
                df_output = False)

scores = evaluate_clusters(scores,  list(result_a['clusters']),
                           list(labels), name='FAMD_Kmeans', 
                           X=matrix_a)
scores

,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125


In [ ]:
# design base models first

In [ ]:
# preprocessing
id_col, cat_col, con_col = get_types(df)
id_col, cat_col, con_col

In [ ]:
"""Model : 1"""

# 1. Cluster based on continuous data only


df_cos = df[con_columns]
result = kmeans_model(df_cos, 5, df_output = False)
result
# print(list(result['clusters'])[:10], y[:10])

In [ ]:
"""Model : 2"""

# 1. Cluster based on mix data with one hot encoding

one_hot_result1 = kmeans_onehot_mix(df, 
                                   cat_columns, 
                                   5, 
                                   scale = None, 
                                   df_output    = False)
print(list(one_hot_result1['clusters'])[:10], y[:10])

In [ ]:
# k prototype result

"""Model : 3"""

k_proc_result = k_prot_model(df, cat_columns, 5)
print(list(k_proc_result['clusters'])[:10], y[:10])

In [ ]:
""" Model : 4"""

y_tr = kmeans_famd(df, cat_columns, 5, 'auto')
print(list(y_tr['clusters'])[:10], y[:10])

In [ ]:
# why not just calculate mean and sd for each grouping, zscore by group, 
# and manually inspect points w abs(z score) > threshold (eg, 2, 3, etc).

# You could also calculate skew by group ((mean - median)/sd). 
# If there are outliers, you'd have large skew values.

In [ ]:
add_freq(df, 'Churn?')

In [ ]:
# UMAP_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
#                         verbose=0, random_state=random_state,
#                         algorithm='auto').fit(embedding_)


dat = umap_reduce(df)
result, emd = dat

UMAP_clusterer = kmeans_model(emd, 
                 no_of_clusters = 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)

In [14]:
def umap_embd_model(df, 
                    cat_columns, 
                    no_of_clusters, 
                    random_state = 32, 
                    max_iter     = 300, 
                    verbose      = 0, 
                    df_output    = False):
    
    
    df             = cobj(df, cat_columns)
    _, emd         = umap_reduce(df)
    
    UMAP_clusterer = kmeans_model(emd, 
                                 no_of_clusters = no_of_clusters, 
                                 random_state = random_state, 
                                 max_iter     = max_iter, 
                                 verbose      = verbose, 
                                 df_output    = df_output)
    
    return UMAP_clusterer, emd

In [15]:
yt, emd = umap_embd_model(df, 
                    cat_col, 
                    no_of_clusters = 4, 
                    random_state = 32, 
                    max_iter     = 300, 
                    verbose      = 0, 
                    df_output    = False)


/Users/monk/Desktop/MixClu/mixclu/embeddings/umap_embd.py:363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical[c] = (numerical[c] - numerical[c].mean())/numerical[c].std(ddof=0)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/umap/umap_.py:1802: UserWarning: gradient function is not yet implemented for dice distance metric; inverse_transform will be unavailable
  warn(


In [16]:
scores = evaluate_clusters(scores,  list(yt['clusters']),
                           labels, name='ump', 
                           X=emd)
scores

,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125
1,ump,0.656788,7170.036770,0.578484,0.081756,0.014814


In [ ]:
scores = evaluate_clusters(scores,  list(UMAP_clusterer['clusters']),
                           labels, name='ump', 
                           X=emd)
scores

In [21]:

# def Mirkin_model(df, 
#                  cat_columns, 
#                  no_of_clusters, 
#                  random_state = 32, 
#                  max_iter = 300, 
#                  verbose =0, 
#                  df_output = False):
    
    
#     df             = cobj(df, cat_columns)
#     continous_data = df.copy()
#     numeric_cols   = continous_data.select_dtypes(include=np.number)
#     cat_cols       = continous_data.select_dtypes(include='object')

#     # numeric process
#     normalized_df = calculate_zscore(continous_data, numeric_cols)
#     normalized_df = normalized_df[numeric_cols.columns]

#     # categorical process
#     cat_one_hot_df, one_hot_cols = one_hot_encode(continous_data, cat_cols)
#     cat_one_hot_norm_df          = calculate_zscore(cat_one_hot_df, one_hot_cols)

#     # Merge DataFrames
#     processed_df = pd.concat([normalized_df, cat_one_hot_norm_df], axis=1)
    
    
#     mirkin_result = kmeans_model(processed_df, 
#                                  no_of_clusters = no_of_clusters, 
#                                  random_state = random_state, 
#                                  max_iter     = max_iter, 
#                                  verbose      = verbose, 
#                                  df_output    = df_output)
    
    
    
#     return processed_df, mirkin_result

In [43]:
a, b = Mirkin_model(df, cat_col, 4)

scores = evaluate_clusters(scores,  list(b['clusters']),
                           labels, name='cat2enca', 
                           X=a)
scores

,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125
1,ump,0.656788,7170.036770,0.578484,0.081756,0.014814
2,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629
3,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629


,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125
1,ump,0.656788,7170.036770,0.578484,0.081756,0.014814
2,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629


In [ ]:
UMAP_clusterer2 = kmeans_model(processed_df, 
                 no_of_clusters = 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)

In [ ]:
scores = evaluate_clusters(scores,  list(UMAP_clusterer2['clusters']),
                           labels, name='cat2enc', 
                           X=processed_df)
scores

In [42]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]


def k_prototype_z_model(df, 
                        cat_columns, 
                        num_of_clusters, 
                        init_method = 'Cao', 
                        random_state= 32, 
                        max_iter    = 300, 
                        n_jobs      = 1, 
                        verbose     = 0, 
                        df_output   = False, 
                        n_init      = 10):
    
    
    z_data              = zscore_preprocessing(df, 
                                               cat_columns, 
                                               cat_col_zscore = False)
    
    categorical_indices = column_index(z_data, cat_columns)
    
    model_result = k_prot_model(z_data, 
                               list(cat_columns), 
                               num_of_clusters, 
                               init_method   = init_method,
                               random_state  = random_state,
                                max_iter     = max_iter,
                                n_jobs       = n_jobs,
                                verbose      = verbose, 
                                df_output    = df_output, 
                                n_init       = n_init)
    
    return model_result
    
    
def Mirkin_model(df, 
                 cat_columns, 
                 no_of_clusters, 
                 random_state = 32, 
                 max_iter = 300, 
                 verbose =0, 
                 df_output = False):
    
    
    
    z_data        = zscore_preprocessing(df, 
                                         cat_columns, 
                                         cat_col_zscore = True)
    
    mirkin_result = kmeans_model(z_data, 
                                 no_of_clusters = no_of_clusters, 
                                 random_state = random_state, 
                                 max_iter     = max_iter, 
                                 verbose      = verbose, 
                                 df_output    = df_output)
    
    
    
    return z_data, mirkin_result

In [32]:
def zscore_preprocessing(df, cat_columns, 
                         cat_col_zscore = True):

    df             = cobj(df, cat_columns)
    continous_data = df.copy()
    numeric_cols   = continous_data.select_dtypes(include=np.number)
    cat_cols       = continous_data.select_dtypes(include='object')
    
    if cat_col_zscore:

        # numeric process
        normalized_df = calculate_zscore(continous_data, numeric_cols)
        normalized_df = normalized_df[numeric_cols.columns]

        # categorical process
        cat_one_hot_df, one_hot_cols = one_hot_encode(continous_data, cat_cols)
        cat_one_hot_norm_df          = calculate_zscore(cat_one_hot_df, one_hot_cols)

        # Merge DataFrames
        processed_df = pd.concat([normalized_df, cat_one_hot_norm_df], axis=1)
    else:
        
        norm_num_cols = calculate_zscore(numeric_cols, numeric_cols)
        processed_df  = pd.concat([norm_num_cols, cat_cols], axis=1)

    return processed_df



In [38]:
ty = k_prototype_z_model(df, cat_col, 4)

In [39]:
scores1 = evaluate_clusters(scores,  list(ty["clusters"]),
                           labels, name='K_Prototype', 
                           X=None)

scores1

,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125
1,ump,0.656788,7170.036770,0.578484,0.081756,0.014814
2,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629
3,K_Prototype,NaN,NaN,NaN,0.467790,0.355819


In [ ]:
mk = k_prot_model(processed_df, 
           list(cat_cols.columns), 
           4, 
           init_method  = 'Cao',
           random_state = 32,
            max_iter    = 300,
            n_jobs       = 1,
            verbose     = 0, 
            df_output   = False, 
            n_init      = 10)

In [ ]:
scores1 = evaluate_clusters(scores,  list(mk["clusters"]),
                           labels, name='K_Prototype', 
                           X=None)

scores1

In [64]:
def kmodes_cont(df, 
                bin_con_columns,
                n_of_clusters,
                bin_bins         = 5, 
                bin_drop_cols    = True, 
                bin_encode       = False, 
                mode_init_method = 'Cao', 
                mode_n_init      = 10, 
                mode_verbose     = 0, 
                mode_random_state= 32, 
                mode_n_jobs      = 1, 
                mode_cat_dissim  = None, 
                mode_df_output   = None):

    recode_df = get_knn_bins(df, bin_con_columns, 
                     bins= bin_bins, 
                     drop_cols=bin_drop_cols, 
                     encode = bin_encode)


    k_mode_result = kmodes_model(recode_df, 
                       n_of_clusters,
                      init         = mode_init_method,
                      n_init       = mode_n_init, 
                      verbose      = mode_verbose,
                      random_state = mode_random_state,
                      n_jobs       = mode_n_jobs,
                      cat_dissim = mode_cat_dissim, 
                      df_output  = mode_df_output)
    
    return k_mode_result

In [61]:
cat_cols = recode_df.select_dtypes(include='object')
categorical_indices = column_index(recode_df, cat_cols.columns)
categorical = list(categorical_indices)

In [65]:
hjk = kmodes_cont(df, con_col, 4)

/Users/monk/Desktop/MixClu/mixclu/embeddings/umap_embd.py:446: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  for col in cols:


New cat columns  Age_value,Overall_value,Potential_value,Value_value,Wage_value,Special_value,International Reputation_value,Weak Foot_value,Skill Moves_value,Height_value,Weight_value,Crossing_value,Finishing_value,HeadingAccuracy_value,ShortPassing_value,Volleys_value,Dribbling_value,Curve_value,FKAccuracy_value,LongPassing_value,BallControl_value,Acceleration_value,SprintSpeed_value,Agility_value,Reactions_value,Balance_value,ShotPower_value,Jumping_value,Stamina_value,Strength_value,LongShots_value,Aggression_value,Interceptions_value,Positioning_value,Vision_value,Penalties_value,Composure_value,Marking_value,StandingTackle_value,GKDiving_value,GKHandling_value,GKKicking_value,GKPositioning_value,GKReflexes_value,Release Clause_value,SlidingTackle_value


In [59]:
yio = kmodes_model(recode_df, 4,
                  init         = 'Cao',
                  n_init       = 10, 
                  verbose      = 0,
                  random_state = 32,
                  n_jobs       = 1,
                  cat_dissim = None, 
                  df_output  = False)

In [66]:
scores = evaluate_clusters(scores,  list(hjk["clusters"]),
                           labels, name='K_mode1', 
                           X=None)

scores

,Algorithm,Silhouette ↑,Calinski_Harabasz ↑,Davis Bouldin ↓,Adjusted_Mutual_Info ↑,Adjusted_Rand_Score ↑
0,FAMD_Kmeans,0.414719,3526.406032,0.760051,0.404003,0.277125
1,ump,0.656788,7170.036770,0.578484,0.081756,0.014814
2,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629
3,cat2enca,0.185227,816.284528,1.682128,0.464976,0.352629
4,K_mode1,NaN,NaN,NaN,0.318991,0.191887
5,K_mode1,NaN,NaN,NaN,0.318991,0.191887
6,K_mode1,NaN,NaN,NaN,0.318991,0.191887
7,K_mode1,NaN,NaN,NaN,0.316035,0.195301
